## Import Dataset

In [1]:
import pandas as pd
import json

In [3]:
file = "/home/braiyenmassora/codex-telkom/resume-parser/src/notebooks/support/university.csv"
parsed_text = {}

def extract_university(text, file):
    df = pd.read_csv(file, header=None)
    universities = [i.lower() for i in df[1]]
    college_name = []
    listex = universities
    listsearch = [text.lower()]

    for i in range(len(listex)):
        for ii in range(len(listsearch)):

            if re.findall(listex[i], re.sub(" +", " ", listsearch[ii])):

                college_name.append(listex[i])

        return college_name
    
univ = extract_university(text)
print(univ)
parsed_text["university"] = univ

NameError: name 'text' is not defined

In [ ]:

    def read_file(file,docx_parser = "tika"):
        """
        file : Give path of resume file
        docx_parser : Enter docx2txt or tika, by default is tika
        """
        # file = "/content/Asst Manager Trust Administration.docx"
        file = os.path.join(file)
        if file.endswith('docx') or file.endswith('doc'):
            if file.endswith('doc') and docx_parser == "docx2txt":
              docx_parser = "tika"
              logging.error("doc format not supported by the docx2txt changing back to tika")
            resume_lines, raw_text = resumeparse.convert_docx_to_txt(file,docx_parser)
        elif file.endswith('pdf'):
            resume_lines, raw_text = resumeparse.convert_pdf_to_txt(file)
        elif file.endswith('txt'):
            with open(file, 'r', encoding='latin') as f:
                resume_lines = f.readlines()

        else:
            resume_lines = None
        resume_segments = resumeparse.segment(resume_lines)
        
        
        full_text = " ".join(resume_lines)

        email = resumeparse.extract_email(full_text)
        phone = resumeparse.find_phone(full_text)
        name = resumeparse.extract_name(" ".join(resume_segments['contact_info']))
        total_exp, text = resumeparse.get_experience(resume_segments)
        university = resumeparse.extract_university(full_text, os.path.join(base_path,'world-universities.csv'))

        designition = resumeparse.job_designition(full_text)
        designition = list(dict.fromkeys(designition).keys())

        degree = resumeparse.get_degree(full_text)
        company_working = resumeparse.get_company_working(full_text)
        
     
        
        return {
            
            "university": university,
           
        }

In [2]:
univ = pd.read_csv("/home/braiyenmassora/codex-telkom/resume-parser/src/notebooks/support/university.csv")
univ.head()

,Rank,University,Town
0,1,Universitas Gadjah Mada,Sleman
1,2,Universitas Indonesia,Depok
2,3,Universitas Diponegoro,Semarang
3,4,Universitas Pendidikan Indonesia,Bandung
4,5,Universitas Negeri Yogyakarta,Sleman


In [3]:
x = univ.drop('Rank', axis=1)
x.head()

,University,Town
0,Universitas Gadjah Mada,Sleman
1,Universitas Indonesia,Depok
2,Universitas Diponegoro,Semarang
3,Universitas Pendidikan Indonesia,Bandung
4,Universitas Negeri Yogyakarta,Sleman


In [ ]:
# convert the dataframe to plain text and save it to a file
with open('output.txt', 'w') as f:
    f.write(x.to_string(index=False))


## Load Anotations

In [ ]:
import os
os.chdir(r'/home/braiyenmassora/codex-telkom/resume-parser/src/notebooks/anotations')

with open('educations.json', 'r') as f:
    data = json.load(f)

In [ ]:
data

In [ ]:
entity_name = "EDUCATIONS"
train_data = data['annotations']
train_data = [tuple(i) for i in train_data]

In [ ]:
train_data

In [ ]:
for i in train_data:
    if i[1]['entities'] == []:
        i[1]['entities'] = (0, 0, entity_name)
    else:
        i[1]['entities'][0] = tuple(i[1]['entities'][0])

In [ ]:
train_data

In [ ]:
model = None
output_dir=("/home/braiyenmassora/codex-telkom/resume-parser/src/notebooks/models")
n_iter=100

In [ ]:
#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

In [ ]:
for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(train_data)
        losses = {}
        for text, annotations in tqdm(train_data):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

In [ ]:
for text, _ in train_data:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk("/home/braiyenmassora/codex-telkom/resume-parser/src/notebooks/models")
    print("Saved model to", output_dir)

## Models Testing

In [ ]:
nlp1 = spacy.load("/home/braiyenmassora/codex-telkom/resume-parser/src/notebooks/models") #load the best model
doc = nlp1("UNIVERSITAS PENDIDIKAN INDONESIA") # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter